---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    univ_df= pd.read_csv("university_towns.txt", sep= "\n", header=None)
    state= ""
    data=[]
    for x in univ_df[0]:
        
        if x.endswith("[edit]"):
            state = x[0:-6]
            continue
        
        if "(" in x:
            i= x.index("(")
            region= x[0:i-1]
            data.append([state, region])
            
        else:
            region= x
            data.append([state, region]) 
    university_df =pd.DataFrame(data, columns=['State', 'RegionName'])
            
    return university_df

In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp_df= (pd.read_excel('gdplev.xls', skiprows=220, parse_cols="E,G", header=None,
                           names=['Quarter','GDP']))
    
    
    for x in range(2,len(gdp_df)):
        
        if (gdp_df.iloc[x-2,1] > gdp_df.iloc[x-1,1])and (gdp_df.iloc[x-1,1] > gdp_df.iloc[x,1]):
                return gdp_df.iloc[x-1,0]
    
         

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp_df= (pd.read_excel('gdplev.xls', skiprows=220, parse_cols="E,G", header=None,
                           names=['Quarter','GDP']))
    
    for x in range(4,len(gdp_df)):
        
        if ((gdp_df.loc[x-4, 'GDP'] > gdp_df.loc[x-3, 'GDP']) and (gdp_df.loc[x-3, 'GDP'] > gdp_df.loc[x-2, 'GDP'])
                and (gdp_df.loc[x-2, 'GDP'] < gdp_df.loc[x-1, 'GDP']) and (gdp_df.loc[x-1, 'GDP'] < gdp_df.loc[x, 'GDP'])):
            
            return gdp_df.iloc[x,0]


In [11]:
def get_recession_bottom():
    
    gdp_df= (pd.read_excel('gdplev.xls', skiprows=220, parse_cols="E,G", header=None,
                           names=['Quarter','GDP']))
    
    start = get_recession_start()
    start_in = gdp_df.index[gdp_df['Quarter'] == start].tolist()[0]
    end = get_recession_end()
    end_in = gdp_df.index[gdp_df['Quarter'] == end].tolist()[0]
    df= gdp_df[start_in: end_in+1]
    df=df.reset_index(drop=True)
    ans= df['GDP'].idxmin()
    
    return  df.iloc[ans,0]

In [65]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    state_df= get_list_of_university_towns()
    housing_df= pd.read_csv("City_Zhvi_AllHomes.csv")
    housing_df['State'] = housing_df['State'].replace(states)
    h_df= pd.DataFrame(housing_df[['State','RegionName']])
    
    for i in range(2000, 2016):
        h_df[str(i)+'q1'] = (housing_df[str(i)+'-'+'01']+housing_df[str(i)+'-'+'02']+housing_df[str(i)+'-'+'03'])/3
        h_df[str(i)+'q2'] = (housing_df[str(i)+'-'+'04']+housing_df[str(i)+'-'+'05']+housing_df[str(i)+'-'+'06'])/3
        h_df[str(i)+'q3'] = (housing_df[str(i)+'-'+'07']+housing_df[str(i)+'-'+'08']+housing_df[str(i)+'-'+'09'])/3
        h_df[str(i)+'q4'] = (housing_df[str(i)+'-'+'10']+housing_df[str(i)+'-'+'11']+housing_df[str(i)+'-'+'12'])/3
        
    h_df['2016q1'] =(housing_df['2016-01']+housing_df['2016-02']+housing_df['2016-03'])/3
    h_df['2016q2'] =(housing_df['2016-04']+housing_df['2016-05']+housing_df['2016-06'])/3
    h_df['2016q3'] =(housing_df['2016-07']+housing_df['2016-08'])/2
    h_df= h_df.set_index(['State','RegionName'])    
    return h_df

In [194]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    from scipy import stats
    
    state_df= get_list_of_university_towns()
    start= get_recession_start()
    end= get_recession_end()
    bottom = get_recession_bottom()

    h_df= convert_housing_data_to_quarters()
    start_in = h_df.columns.get_loc(start)
    end_in = h_df.columns.get_loc(end)

    df= h_df.iloc[:,start_in:end_in+1]
    df=df.reset_index()
    univ_df=pd.DataFrame(columns = df.columns)
    nu_df= pd.DataFrame(columns=df.columns)
    
    for ind,x  in df['RegionName'].iteritems():
        ins= df.index[x==state_df['RegionName']].tolist()
        if ins ==[]:
            nu_df= nu_df.append(df.iloc[ind], ignore_index=True)
            continue
            
        for i in ins:
            univ_df= univ_df.append(df.iloc[i], ignore_index=True)
    
    univ_df['Price Ratio']= univ_df[bottom]-univ_df[start]
    nu_df['Price Ratio']= nu_df[bottom]- nu_df[start]
    
    s,p = ttest_ind(univ_df['Price Ratio'], nu_df['Price Ratio'], nan_policy = 'omit')
    
    if p<0.01 : different = True
    else: different= False
    
    better = "non-university town" if univ_df["Price Ratio"].mean()< nu_df["Price Ratio"].mean() else "university town" 
    return (different,p,better)